## Dataset 


PASCAL VOC2012 이용

http://host.robots.ox.ac.uk/pascal/VOC/voc2012/

In [3]:
import os
import urllib.request
import zipfile
import tarfile

In [5]:
# data 폴더 작성
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

# weights 폴더 작성
weights_dir = "./weights/"
if not os.path.exists(weights_dir):
    os.mkdir(weights_dir)

In [6]:
# download
url = "http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar"
target_path = os.path.join(data_dir, "VOCtrainval_11-May-2012.tar") 

if not os.path.exists(target_path):
    urllib.request.urlretrieve(url, target_path)
    
    tar = tarfile.TarFile(target_path)  
    tar.extractall(data_dir)  
    tar.close() 

In [7]:
# pretrained된 SSD용 VGG16 파라미터 다운로드
url = "https://s3.amazonaws.com/amdegroot-models/vgg16_reducedfc.pth"
target_path = os.path.join(weights_dir, "vgg16_reducedfc.pth") 

if not os.path.exists(target_path):
    urllib.request.urlretrieve(url, target_path)

In [8]:
# pretrained된 SSD300 모델 다운로드
url = "https://s3.amazonaws.com/amdegroot-models/ssd300_mAP_77.43_v2.pth"
target_path = os.path.join(weights_dir, "ssd300_mAP_77.43_v2.pth") 

if not os.path.exists(target_path):
    urllib.request.urlretrieve(url, target_path)


In [9]:
import os.path as osp
import random
# 파일이나 텍스트에서 XML을 읽고, 가공하고 저장하는 라이브러리
import xml.etree.ElementTree as ET

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.utils.data as data

%matplotlib inline

## 1. 이미지 & annotation 데이터의 파일 경로 리스트 작성

목적 : 모든 이미지 데이터, annotation 파일의 경로를 리스트형 변수로 작성

-> train_img_list, train_anno_list, val_img_list, val_anno_list 출력

- Object detection은 이미지 데이터와 대응되는 annotation 데이터를 데이터셋에서 함께 처리

- 파일, 폴더 이름에 정답 클래스명이 포함

- 객체 위치(bounding box), 정답 클래스 정보는 annotation 데이터로 제공

In [10]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [15]:
def make_datapath_list(rootpath):
    # 이미지, annotation 파일 경로
    imgpath_template = osp.join(rootpath, 'JPEGImages', '%s.jpg')  # ex) 2007_000027.jpg
    annopath_template = osp.join(rootpath, 'Annotations', '%s.xml')
    
    # 파일 ID
    train_id_names = osp.join(rootpath + 'ImageSets/Main/train.txt')  # ex) 2007_000027
    val_id_names = osp.join(rootpath + 'ImageSets/Main/val.txt')
    
    # train용
    train_img_list = []
    train_anno_list = []

    for line in open(train_id_names):
        file_id = line.strip()
        img_path = (imgpath_template % file_id)
        anno_path = (annopath_template % file_id) 
        
        train_img_list.append(img_path) 
        train_anno_list.append(anno_path) 

    # val용
    val_img_list = []
    val_anno_list = []

    for line in open(val_id_names):
        file_id = line.strip() 
        img_path = (imgpath_template % file_id)  
        anno_path = (annopath_template % file_id) 

        val_img_list.append(img_path)  
        val_anno_list.append(anno_path)  

    return train_img_list, train_anno_list, val_img_list, val_anno_list

In [19]:
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath)

# 각 이미지 파일에 대응되는 annotation 파일 존재
print(train_img_list[0:2])
print(train_anno_list[0:2])

['./data/VOCdevkit/VOC2012/JPEGImages/2008_000008.jpg', './data/VOCdevkit/VOC2012/JPEGImages/2008_000015.jpg']
['./data/VOCdevkit/VOC2012/Annotations/2008_000008.xml', './data/VOCdevkit/VOC2012/Annotations/2008_000015.xml']


## 2. XML 형식 annotation 데이터를 리스트 형식으로 

목적 : xml 형식의 annotation data의 bbox 좌표, 정답 클래스 정보를 리스트 형식으로 저장

-> [[xmin, ymin, xmax, ymax, label_ind], ... ]  : 한 장의 이미지에 대한 결과

![image](https://user-images.githubusercontent.com/44194558/147519071-e187c42d-3206-4cae-93f3-db20ae86050d.png)

In [21]:
class Anno_xml2list(object):

    def __init__(self, classes):
        self.classes = classes

    def __call__(self, xml_path, width, height):
        ret = []
        xml = ET.parse(xml_path).getroot()
        
        # 하나의 이미지에 존재하는 모든 객체의 정보를 처리
        for obj in xml.iter('object'):
            difficult = int(obj.find('difficult').text)
            if difficult == 1:
                continue
            
            bndbox = []

            name = obj.find('name').text.lower().strip()
            bbox = obj.find('bndbox')
            
            # annotation 파일의 bbox 좌표를 0~1로 정규화
            pts = ['xmin', 'ymin', 'xmax', 'ymax']

            for pt in (pts):
                cur_pixel = int(bbox.find(pt).text) - 1  # VOC는 원점이 (1, 1)

                if pt == 'xmin' or pt == 'xmax':
                    cur_pixel /= width  # 이미지의 폭으로 나눠서 정규화

                else:
                    cur_pixel /= height  # 이미지의 높이로 나눠서 정규화

                bndbox.append(cur_pixel)

            label_idx = self.classes.index(name)
            bndbox.append(label_idx)

            ret += [bndbox]

        return np.array(ret)  # [[xmin, ymin, xmax, ymax, label_ind], ... ], 객체가 여러개 있을 경우 여러 개의 내부 리스트 존재

In [22]:
# class 정의
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']

transform_anno = Anno_xml2list(voc_classes)

In [24]:
# 확인용 예시
ind = 1
image_file_path = val_img_list[ind]
img = cv2.imread(image_file_path)  # [높이][폭][BGR]
height, width, channels = img.shape 

transform_anno(val_anno_list[ind], width, height)

array([[ 0.09      ,  0.03003003,  0.998     ,  0.996997  , 18.        ],
       [ 0.122     ,  0.56756757,  0.164     ,  0.72672673, 14.        ]])

## 3. 이미지, annotation 전처리

이미지와 bbox 정보 변환 수행. 학습, 추론 시에 다르게 작동하도록.

 - 학습 시에는 DataTransform으로 데이터 augmentation 수행
   - Augmentation에서 이미지의 색과 크기가 변경되기 때문에, 이에 대응되는 annotation의 bbox 좌표 정보도 같이 변경되어야 함

 - 추론 시에는 이미지의 크기를 변경하고 색상 정보의 평균값을 뺌

In [ ]:
from utils.data_augumentation import Compose, ConvertFromInts, ToAbsoluteCoords, PhotometricDistort, Expand, RandomSampleCrop, RandomMirror, ToPercentCoords, Resize, SubtractMeans

class DataTransform(): 
    
    def __init__(self, input_size, color_mean):
        self.data_transform = {
            'train': Compose([
                ConvertFromInts(),  # int를 float32로 변환
                ToAbsoluteCoords(),  # 정규화된 annotation 데이터
                PhotometricDistort(),  # 이미지의 색 등을 임의로 변화시킴
                Expand(color_mean),  # 이미지의 캔버스 확대
                RandomSampleCrop(),  # 이미지 내의 특정 부분을 무작위로 추출
                RandomMirror(),  # 이미지를 반전
                ToPercentCoords(),  # annotation 데이터를 0-1로 정규화
                Resize(input_size),  # 이미지 크기를 input_size × input_size로 변형
                SubtractMeans(color_mean)  # BGR 색상의 평균값을 뺀다
            ]),

            'val': Compose([
                ConvertFromInts(),  # int를 float로 변환
                Resize(input_size),  # 이미지 크기를 input_size × input_size로 변형
                SubtractMeans(color_mean)  # BGR 색상의 평균값을 뺀다
            ])
        }

    def __call__(self, img, phase, boxes, labels):
        return self.data_transform[phase](img, boxes, labels)

In [ ]:
# 확인용 예시
image_file_path = train_img_list[0]
img = cv2.imread(image_file_path)  # [높이][폭][색BGR]
height, width, channels = img.shape 

# 2. annotation을 리스트로
transform_anno = Anno_xml2list(voc_classes)
anno_list = transform_anno(train_anno_list[0], width, height)

# 3. 원본 이미지
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

# 4. 전처리 클래스 
color_mean = (104, 117, 123)  # (BGR) 색상의 평균값
input_size = 300  # 이미지 input 사이즈를 300×300으로
transform = DataTransform(input_size, color_mean)

# 5. train 이미지 표시
phase = "train"
img_transformed, boxes, labels = transform(
    img, phase, anno_list[:, :4], anno_list[:, 4])
plt.imshow(cv2.cvtColor(img_transformed, cv2.COLOR_BGR2RGB))
plt.show()


# 6. val 이미지 표시
phase = "val"
img_transformed, boxes, labels = transform(
    img, phase, anno_list[:, :4], anno_list[:, 4])
plt.imshow(cv2.cvtColor(img_transformed, cv2.COLOR_BGR2RGB))
plt.show()

## 4. Dataset

목적 : VOCDataset 클래스 생성

 - 앞에서 정의한 클래스 활용

In [ ]:
class VOCDataset(data.Dataset):
    """
    Attributes
    ----------
    img_list : 리스트
        이미지 경로를 저장한 리스트
    anno_list : 리스트
        annotation 경로를 저장한 리스트
    phase : 'train' or 'test'
        train/test 설정
    transform : object
        전처리 클래스의 인스턴스
    transform_anno : object
        xml annotation을 리스트로 변환
    """

    def __init__(self, img_list, anno_list, phase, transform, transform_anno):
        self.img_list = img_list
        self.anno_list = anno_list
        self.phase = phase  
        self.transform = transform  
        self.transform_anno = transform_anno  

    def __len__(self):
        return len(self.img_list)
    
    # 전처리 완료된 이미지 tensor, annotation
    def __getitem__(self, index):
        im, gt, h, w = self.pull_item(index)
        return im, gt
    
    # 전처리 완료된 이미지 tensor, annotation, h, w
    def pull_item(self, index):
        # 하나의 이미지 읽기
        image_file_path = self.img_list[index]
        img = cv2.imread(image_file_path)  # [높이][폭][색BGR]
        height, width, channels = img.shape  
        
        # 개별 이미지에 대응되는 annotation
        anno_file_path = self.anno_list[index]
        anno_list = self.transform_anno(anno_file_path, width, height)
        
        # 전처리
        img, boxes, labels = self.transform(
            img, self.phase, anno_list[:, :4], anno_list[:, 4])
        
        # BGR -> RGB
        # (h, w, c) -> (c, h, w)
        img = torch.from_numpy(img[:, :, (2, 1, 0)]).permute(2, 0, 1)

        gt = np.hstack((boxes, np.expand_dims(labels, axis=1)))

        return img, gt, height, width

In [ ]:
# 확인용 예시
color_mean = (104, 117, 123)  # (BGR) 색의 평균값
input_size = 300  

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset.__getitem__(1)

## DataLoader

Dataset에서 꺼내는 annotation 데이터의 크기는 이미지 마다 다름. (이미지 내에 존재하는 객체의 수가 다르기 때문에)

In [26]:
def od_collate_fn(batch):

    targets = []
    imgs = []
    for sample in batch:
        imgs.append(sample[0])  # sample[0]는 이미지
        targets.append(torch.FloatTensor(sample[1]))  # sample[1]는 annotation gt

    # imgs는 미니 배치 크기의 리스트
    # 리스트의 개별 요소는 torch.Size([3, 300, 300]) - 300x300 크기의 칼라 이미지
    # imgs 리스트를 torch.Size([batch_num, 3, 300, 300])의 텐서로 변환
    imgs = torch.stack(imgs, dim=0)

    # targets는 annotation gt의 리스트
    # 리스트의 크기는 미니 배치의 크기
    # targets의 개별 요소는 [n, 5] 크기 (bbox 좌표 4개 + 정답 클래스 1개)
    # n은 이미지마다 다름 (이미지 내에 존재하는 객체의 수)

    return imgs, targets


In [ ]:
batch_size = 4

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)

# 사전형 변수에 정리
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

# 동작 확인
batch_iterator = iter(dataloaders_dict["val"])  # 반복자로 변환
images, targets = next(batch_iterator)  # 첫 번째 요소를 추출
print(images.size())  # torch.Size([4, 3, 300, 300])
print(len(targets))
print(targets[1].size())  

In [ ]:
print(train_dataset.__len__())
print(val_dataset.__len__())